In [ ]:
import sys
sys.path.append('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/')

from util.util_train import *
from util.util_eval import * 

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.preprocessing import label_binarize

import pandas as pd 
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)


In [ ]:
wd = Path('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/validation')
type_path = Path('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/util/type_info_PDO_3chl.json')

## 1. Choose best model

In [ ]:
pn = wd.joinpath('data/checkpoint9/stats0.json') 
with open(pn) as f:
    data = json.load(f)
df = pd.DataFrame(data).T
df.filter(regex='valid.*dice').plot(xticks=range(0,51),figsize = (20,10))
plt.title('Dice Scores for Validation Split')
plt.savefig(wd.joinpath('output/fig/phase0_val_dice.pdf'))

In [ ]:
pn = wd.joinpath('data/checkpoint9/stats1.json') 
with open(pn) as f:
    data = json.load(f)
df = pd.DataFrame(data).T
df.filter(regex='valid.*dice').plot(xticks=range(0,51),figsize = (20,10))
plt.title('Dice Scores for Validation Split')
plt.savefig(wd.joinpath('output/fig/phase1_val_dice.pdf'))

In [ ]:
df.iloc[[31],:]   #epoch32

##### Best model is phase2 epoch 32, use it to run predictions

## 2. Generate metrics for training split 

### 2.1 Gen train_true_gson

In [ ]:
###### extract images from pathes
indir  = wd.joinpath('data/train_val_tile/train2')
outdir = wd.joinpath('data/train_val_tile/train2_patch_im')
extract_image (outdir,indir)

###### extract masks from pathes       
outdir = wd.joinpath('data/train_val_tile/train2_patch_mask')
extract_mask (outdir,indir)

###### convert true mask to true gson
celltypes = ['background','normal','tumor']
colors=[[255,255,255],[255, 0, 0],[0, 255, 0]]

indir  = wd.joinpath('data/train_val_tile/train2_patch_mask')
outdir = wd.joinpath('data/train_val_tile/train2_patch_true_gson')
hvmask2gson (outdir,indir,celltypes,colors,resize=False)

### 2.2 Gen train_pred_gson: run trained model to predict cell types from training tiles  

In [ ]:
###### json2gson
indir  = wd.joinpath('data/train_val_tile/train2_patch_im_json_chk9ep32')
outdir = wd.joinpath('data/train_val_tile/train2_patch_pred_gson')
df = json2gson(outdir,type_path,indir)

### 2.3 Get train metrics: precision etc

In [ ]:
gs_true_dir = wd.joinpath('data/train_val_tile/train2_patch_true_gson')
gs_pred_dir = wd.joinpath('data/train_val_tile/train2_patch_pred_gson')
celltypes = ['normal','tumor']
shape = (256,256)
df = gen_ypred_ytrue(gs_true_dir,gs_pred_dir,celltypes,shape)

#### 2.3.1 detection metrics

In [ ]:
df['dtc_ytrue'] = np.where(df.ytrue != 'NA_true', 'cell_in_true', 'cell_not_in_true')
df['dtc_ypred'] = np.where(df.ypred != 'NA_pred', 'cell_in_true', 'cell_not_in_true')

ytrue = df.dtc_ytrue
ypred = df.dtc_ypred

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['cell_in_true','cell_not_in_true']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- perf report
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report

#### 2.3.2  classification metrics

In [ ]:
df = df.query("ypred != 'NA_pred' and ytrue != 'NA_true'")
print(df.ypred.unique())
print(df.ytrue.unique())

In [ ]:
ytrue = df.ytrue
ypred = df.ypred

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd.joinpath('output/fig/train_tile_detection_roc.pdf'))

#-------- perf report
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report

## 3. Val data split metrics

### 2.1  Gen val_true_gson 

In [ ]:
###### extract images from pathes
indir  = wd.joinpath('data/train_val_tile/val2')
outdir = wd.joinpath('data/train_val_tile/val2_patch_im')
extract_image (outdir,indir)

###### extract masks from pathes       
outdir = wd.joinpath('data/train_val_tile/val2_patch_mask')
extract_mask (outdir,indir)

###### convert true mask to true gson
celltypes = ['background','normal','tumor']
colors=[[255,255,255],[255, 0, 0],[0, 255, 0]]

indir  = wd.joinpath('data/train_val_tile/val2_patch_mask')
outdir = wd.joinpath('data/train_val_tile/val2_patch_true_gson')
hvmask2gson (outdir,indir,celltypes,colors,resize=False)

### 2.2  Gen val_pred_gson

In [ ]:
###### json2gson
indir  = wd.joinpath('data/train_val_tile/val2_patch_im_json_chk9ep32')
outdir = wd.joinpath('data/train_val_tile/val2_patch_pred_gson')
df = json2gson(outdir,type_path,indir)

###  2.3 Get eval metrics: precision etc

In [ ]:
gs_true_dir = wd.joinpath('data/train_val_tile/val2_patch_true_gson')
gs_pred_dir = wd.joinpath('data/train_val_tile/val2_patch_pred_gson')
celltypes = ['normal','tumor']
shape = (256,256)

df = gen_ypred_ytrue(gs_true_dir,gs_pred_dir,celltypes,shape)

In [ ]:
df['dtc_ytrue'] = np.where(df.ytrue != 'NA_true', 'cell_in_true', 'cell_not_in_true')
df['dtc_ypred'] = np.where(df.ypred != 'NA_pred', 'cell_in_true', 'cell_not_in_true')

ytrue = df.dtc_ytrue
ypred = df.dtc_ypred

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['cell_in_true','cell_not_in_true']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- perf report
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report

In [ ]:
df = df.query("ypred != 'NA_pred' and ytrue != 'NA_true'")
print(df.ypred.unique())
print(df.ytrue.unique())

In [ ]:
ytrue = df.ytrue
ypred = df.ypred

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)


#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd.joinpath('val_tile_classification_roc.pdf'))

#-------- perf report
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report